<a href="https://colab.research.google.com/github/Naoki0424/my_document/blob/main/20211107_Youtube%E3%83%81%E3%83%A3%E3%83%B3%E3%83%8D%E3%83%ABID%E5%8F%96%E5%BE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YoutubeチャンネルのID取得スクリプト

## Youtube Api Dataの設定

In [ ]:
from apiclient.discovery import build
from apiclient.errors import HttpError
import pandas as pd
 
# Youtube data api を利用する上での情報を定義する
DEVELOPER_KEY = ''
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'
 
youtube = build(
    YOUTUBE_API_SERVICE_NAME, 
    YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY
    )

## 情報取得

In [ ]:
# 任意の検索条件でチャンネル情報を取得する
def get_video_inoformation(part, q, order, type, maxResults=5):
    # データ取得件数(4の場合は200チャンネル)
    num =6
    result = []
    search_response = youtube.search().list(
        q=q,
        part=part,
        maxResults=maxResults,
        type=type,
        regionCode='JP'
        )
    output = search_response.execute()

    for i in range(num):
        result = result + output['items']
        search_response = youtube.search().list_next(search_response, output)
        output = search_response.execute()
 
    return result

In [ ]:
# 対象のチャンネルの詳細情報を取得する
def get_video_detail(channel_id):
    search_response = youtube.channels().list(
        part='snippet,statistics',
        id=channel_id,
        maxResults=50
    ).execute()

    return search_response

In [ ]:
# video = get_video_inoformation(part='id', q='アラサー 孤独 酒', order='viewCount', type='channel', maxResults=50)
video = get_video_inoformation(part='id,snippet', q='Vlog 社会人 勉強', order='viewCount', type='channel', maxResults=50)


In [ ]:
#各動画毎に一意のvideoIdを取得
channelId_list = []
for result in video:
  if result['id']['channelId'] not in channelId_list:
    channelId_list.append(result['id']['channelId'])

print(len(channelId_list))

# n  = int(len(channelId_list)/50) + 1
n = 50
print(n)

channelIds = [channelId_list[i: i + n] for i in range(0, len(channelId_list), n)]

print(len(channelIds))

for i in range(len(channelIds)):
  channelIds[i] = ",".join(channelIds[i])

print(len(channelIds))
print(channelIds[0])
print(type(channelIds[1]))

290
50
6
6
UCzWiCGpjFWj3Z-gfQO_Fe5w,UClKQXkWzmHxAfmYGwLMHk2A,UCqbblsirstzDwcebju5xYsw,UCk7954B58JqcpCmgDRP7W9w,UCRK6ULLW1w-ja-jH1dXrGeQ,UCDlJYL8EoAQB4M8iQjMOoAQ,UCkD8x5Mc_17yhrhO_Dz4sJg,UCKb-pvZM9-ax6FHd3F1cV7Q,UCYRtDf3MOXRtOM_ZzRTb-Lw,UCs2jEH79N6jxaKQwtsDz8KA,UCZqdnp83yehIAmnXrKABQLg,UCFKRJcgr8Zg7NQJC-YKQsIw,UCSeI03oEtgpFNV8Gnaqj9VA,UCvTd1VHeod06mxpoLnFGGyQ,UCoxI3XyAd87VvzvVE4uGleg,UCPUoof1xbSmjOV5SxR4YiaQ,UCX_CNnukpbQHTO_ET_3MITQ,UCxGDN1EwNd5ukVegZ2BPYMw,UCN3nvWDEFp1J5Xa8uMYE7Rg,UC3WwJoyeNyy4zpKW9y-Vcdg,UCXd6yOXkYoQuYGWL-E_l4oQ,UCpHK3lhpKxgC-xiwAANRPIQ,UCAWYHCV3mCr54npO3Q-WJrw,UCxgIVsWay52aBTCS9owWffQ,UCVTd4TPjqGCF8FFwKuXTAxg,UC_F-t_3B1GwGsa__t69W7bw,UCZDCPOeJLm6Pcx2ou9nCMOg,UC3KAbIaJear-m5kmT6bvnjg,UCqTMpFHg8E2bdI-SzjLzJCA,UCgy1-XhSS3IvjkbRnoUILCA,UCT_Uux1pxZwvVCM6DJ1LxZw,UCtxI5MfTI1XUgENVs-qRhRg,UC84jYNEG6_bIJICWZno-MjQ,UCeodH3Dl-fuqT6V1vNJcRww,UCotoNbXZpQ8yBFaVK28C2Ng,UCKBj3gpS5puaLF-DpVX1XLg,UCw1wTCNndKBVdtyVvco3HRg,UCc6Cpq-bqM9ggEifg5Ou6tw,UCg2qnwl8arSj5eGkJiyCC0w,UCHKn-i_V7eMUo

In [ ]:
video_detail = []

for i in range(len(channelIds)):
  video_detail = video_detail + get_video_detail(channelIds[i])['items']

In [ ]:
def get_subscriberCount(data):
  if data['statistics']['hiddenSubscriberCount']:
    return '非公開'

  return data['statistics']['subscriberCount']

In [ ]:
list = []
print(len(video_detail))
for result in video_detail:
  list.append({'チャンネルID':result['id'], 'チャンネル名': result['snippet']['title'], '開始日':result['snippet']['publishedAt'][:10], '投稿本数':result['statistics']['videoCount'], 'チャンネル登録者数':get_subscriberCount(result)})
  # print(result['snippet']['title'], result['snippet']['publishedAt'], result['statistics']['videoCount'], get_subscriberCount(result))

df = pd.DataFrame(list)


290


In [ ]:
df

,チャンネルID,チャンネル名,開始日,投稿本数,チャンネル登録者数
0,UC3KAbIaJear-m5kmT6bvnjg,こもりん_外資リーマンの勉強・筋トレVlog,2021-01-31,55,693
1,UCRK6ULLW1w-ja-jH1dXrGeQ,RaysVlog / 社会人勉強VLOG,2021-05-07,20,非公開
2,UCxgIVsWay52aBTCS9owWffQ,ぽんすけぶろぐ,2016-03-25,39,771
3,UCXd6yOXkYoQuYGWL-E_l4oQ,ろくまる 30代社会人・子持ちの勉強系Vlog,2021-07-04,6,3
4,UCtxI5MfTI1XUgENVs-qRhRg,Chiharu VLOG / 社会人ルーティン,2021-05-08,35,35
...,...,...,...,...,...
285,UCw3RCydc5_K74047gunkeaw,れのれらTV / RenoRera TV,2015-02-21,1787,176000
286,UCrcM9cvN5MlTPypnNFR4TnA,死ぬまで生きる問題チャンネル,2017-10-29,556,非公開
287,UCF-NghPcqBfeU85LSgU4s0w,Nest,2014-07-17,48,773
288,UC7PcfykDBoh4GMd_lQc8qkQ,ゆうチャンネル,2020-12-11,218,3560


## CSVで保存

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
df.to_csv('/content/gdrive/My Drive/Colab Notebooks/work/youtube_chanel.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
